In [1]:
from rule_generation import * 

### Carregando a base de dados de interesse

Mude a base de dados de acordo com o nome sorteado.

In [2]:
df_name = 'a2212763_data.csv'
df = pd.read_csv(df_name)
df = df.drop(axis=1, labels=['Date'])
df['Volume'] = df['Volume'].apply(lambda x: x/df['Volume'].max())
df_Close = df['Close'].to_frame()
df_High = df['High'].to_frame()
df_Low = df['Low'].to_frame()
df_Open = df['Open'].to_frame()
df_Volume = df['Volume'].to_frame()
df_Close

,Close
0,1.019277
1,1.000514
2,0.955484
3,0.978468
4,0.939536
...,...
278,0.359941
279,0.345231
280,0.342980
281,0.349584


Transformando a série temporal em uma base de dados com lags.

In [3]:
num_lags = 30  # Adjust this value based on your needs

# Create lagged features
for lag in range(1, (num_lags + 1)):
    df_Close[f"lag_{lag}"] = df_Close["Close"].shift(lag)

# Remove rows with NaN values created by shifting
df_Close = df_Close.dropna()

df_Close

,Close,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7,lag_8,lag_9,...,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30
30,0.758008,0.807729,0.771611,0.745343,0.791311,0.826491,0.849007,0.830712,0.863548,0.884655,...,0.854166,0.819924,0.896382,0.938128,0.988788,0.939536,0.978468,0.955484,1.000514,1.019277
31,0.793187,0.758008,0.807729,0.771611,0.745343,0.791311,0.826491,0.849007,0.830712,0.863548,...,0.839625,0.854166,0.819924,0.896382,0.938128,0.988788,0.939536,0.978468,0.955484,1.000514
32,0.760822,0.793187,0.758008,0.807729,0.771611,0.745343,0.791311,0.826491,0.849007,0.830712,...,0.929685,0.839625,0.854166,0.819924,0.896382,0.938128,0.988788,0.939536,0.978468,0.955484
33,0.756132,0.760822,0.793187,0.758008,0.807729,0.771611,0.745343,0.791311,0.826491,0.849007,...,0.952201,0.929685,0.839625,0.854166,0.819924,0.896382,0.938128,0.988788,0.939536,0.978468
34,0.705473,0.756132,0.760822,0.793187,0.758008,0.807729,0.771611,0.745343,0.791311,0.826491,...,0.909047,0.952201,0.929685,0.839625,0.854166,0.819924,0.896382,0.938128,0.988788,0.939536
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.359941,0.368797,0.374651,0.386509,0.374651,0.399117,0.389211,0.382607,0.393264,0.370598,...,0.308307,0.309058,0.313260,0.292696,0.303804,0.324067,0.331272,0.338176,0.332773,0.343280
279,0.345231,0.359941,0.368797,0.374651,0.386509,0.374651,0.399117,0.389211,0.382607,0.393264,...,0.330821,0.308307,0.309058,0.313260,0.292696,0.303804,0.324067,0.331272,0.338176,0.332773
280,0.342980,0.345231,0.359941,0.368797,0.374651,0.386509,0.374651,0.399117,0.389211,0.382607,...,0.327220,0.330821,0.308307,0.309058,0.313260,0.292696,0.303804,0.324067,0.331272,0.338176
281,0.349584,0.342980,0.345231,0.359941,0.368797,0.374651,0.386509,0.374651,0.399117,0.389211,...,0.336525,0.327220,0.330821,0.308307,0.309058,0.313260,0.292696,0.303804,0.324067,0.331272


### Criando a variável linguística para cada um dos lags do problema. 

Nessa etapa, crie e modifique as variáveis linguísticas (e seus respectivos conjuntos Fuzzy) para chegar a uma melhor resposta. 

Abaixo tem uma ideia de como criar essa lista de variáveis linguísticas de uma forma enxuta e rápida. Contudo, tente você também fazer suas adaptações.

In [4]:
min_value = min(df_Close.min())
max_value = max(df_Close.max())

num_fuzzy_sets = 3

variable_list = [ctrl.Antecedent(np.arange(min_value - 1, max_value + 1, 1), name) if name != 'Close' else ctrl.Consequent(np.arange(min_value - 1, max_value + 1, 1), 'target') for name in df_Close.columns]
for variable in variable_list:
    variable.automf(num_fuzzy_sets)
variable_list.reverse()
variable_list

[Antecedent: lag_30,
 Antecedent: lag_29,
 Antecedent: lag_28,
 Antecedent: lag_27,
 Antecedent: lag_26,
 Antecedent: lag_25,
 Antecedent: lag_24,
 Antecedent: lag_23,
 Antecedent: lag_22,
 Antecedent: lag_21,
 Antecedent: lag_20,
 Antecedent: lag_19,
 Antecedent: lag_18,
 Antecedent: lag_17,
 Antecedent: lag_16,
 Antecedent: lag_15,
 Antecedent: lag_14,
 Antecedent: lag_13,
 Antecedent: lag_12,
 Antecedent: lag_11,
 Antecedent: lag_10,
 Antecedent: lag_9,
 Antecedent: lag_8,
 Antecedent: lag_7,
 Antecedent: lag_6,
 Antecedent: lag_5,
 Antecedent: lag_4,
 Antecedent: lag_3,
 Antecedent: lag_2,
 Antecedent: lag_1,
 Consequent: target]

### Divisão da base de dados (treino e teste)

In [5]:
# Split between train and test dataframes
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_Close, test_size=0.1, shuffle=False)
train_df.isnull().sum()

Close     0
lag_1     0
lag_2     0
lag_3     0
lag_4     0
lag_5     0
lag_6     0
lag_7     0
lag_8     0
lag_9     0
lag_10    0
lag_11    0
lag_12    0
lag_13    0
lag_14    0
lag_15    0
lag_16    0
lag_17    0
lag_18    0
lag_19    0
lag_20    0
lag_21    0
lag_22    0
lag_23    0
lag_24    0
lag_25    0
lag_26    0
lag_27    0
lag_28    0
lag_29    0
lag_30    0
dtype: int64

### Criando o sistema Fuzzy usando o método Wang-Mendel

In [ ]:
# Create fuzzy system with reduced variables (avoids interdependent/cyclic rules)
fuzzy_system = create_fuzzy_system(train_df, variable_list, 1)

d:\puc\Fuzzy 2025-2\trabalho_1\rule_generation.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label, p = select_higher_membership(list_variables[j], row[j])
d:\puc\Fuzzy 2025-2\trabalho_1\rule_generation.py:103: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label, p = select_higher_membership(list_variables[idx], row[idx])


Generated Wang-Mendel Rules:
IF good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good THEN good  [degree=0.0000]
IF average, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good THEN good  [degree=0.0000]
IF average, average, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good THEN good  [degree=0.0000]
IF average, average, average, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good, good THEN good  [degree=0.0000]
IF average, average, average, average, good, good, good, good, good, good, good, good, good, good, good, good, good, 

### Fazendo a previsão para o conjunto de treino e de teste.

In [7]:
def predict(fuzzy_sim : ctrl.ControlSystemSimulation, df):
    df_copy = df.copy()
    df_copy.loc[:, 'predict'] = np.nan
    # Create a loop to predict the values 
    for i in range(len(df_copy)):
        l = df_copy.iloc[i,:].shape[0] - 2  # Number of lagged variables
        for k in range(l):
            fuzzy_sim.input[f'lag_{l - k}'] = df_copy.iloc[i, k]
        try:
            fuzzy_sim.compute()
            out = fuzzy_sim.output['target']
        except Exception as e:
            out = df_copy.iloc[i, k]
        df_copy.iloc[i, l + 1] = out 
    return df_copy 

In [ ]:
fuzzy_sim = ctrl.ControlSystemSimulation(fuzzy_system)

predict_train_df = predict(fuzzy_system=fuzzy_system, df=train_df)
predict_train_df

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [ ]:
predict_test_df = predict(fuzzy_system=fuzzy_system, df=test_df)
predict_test_df

In [ ]:
import matplotlib.pyplot as plt 

plt.figure(figsize=(20, 10))
plt.plot(predict_train_df['Close'], label='Training data', linewidth=3)
plt.plot(predict_train_df['predict'], label='Training predict', linewidth=3)
plt.plot(predict_test_df['Close'], label='Test data', linewidth=3)
plt.plot(predict_test_df['predict'], label='Test predict', linewidth=3)
plt.legend()
plt.grid()

plt.show()